<a href="https://colab.research.google.com/github/karaage0703/colab-notebooks/blob/main/code_cooker_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Cooker
Simple data analysis notebook on google colab like Advanced Data Analysis(Code Interpreter).

Reference: [claude_manim.ipynb](https://colab.research.google.com/github/wmoto-ai/claude-manim-notebook/blob/main/claude_manim.ipynb#scrollTo=mF8X8Q3LyZc9)

Install packages

In [ ]:
!pip -qqq install anthropic

Import libraries and setup api

In [ ]:
import os
from google.colab import userdata
import anthropic
import random
import time
from IPython.core.magic import register_cell_magic

os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

Define function

In [ ]:
@register_cell_magic
def _(line, cell):
    system_prompt = f"""あなたはJupyterNotebookを使うPythonデータサイエンティストです。
    入力された指示をもとに、データ分析を実施してください。
    限界を超えてがんばってください。うまくいったらチップあげます。
    """

    user_prompt = cell
    filename = line

    if filename != "":
        system_prompt += f"""
        # 入力データ:ファイルパス
        {filename}
        なお、データはUTF-8で読み込んでください。
        """

    messages = [
        {"role": "user", "content": user_prompt}
    ]

    prompt_w_history.extend(messages)

    retry_count = 0
    while retry_count < 3:
        try:
            response = client.messages.create(
                max_tokens=2048,
                messages=prompt_w_history,
                model="claude-3-opus-20240229",
                system=system_prompt
            )
        except Exception as e:
            print(e)
            time.sleep(3)
            continue

        assistant_response = response.content[0].text

        print("APIからの応答:", assistant_response)

        python = ""
        if "```python" in assistant_response and "```" in assistant_response:
            python = assistant_response.split("```python")[1].split("```")[0]

        print("抽出されたPythonコード:", python)

        try:
            exec(python)
            prompt_w_history.append({"role": "assistant", "content": str(assistant_response)})
            break
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            prompt_w_history.append({"role": "assistant", "content": str(assistant_response)})
            prompt_w_history.append({"role": "user", "content": f"エラーが発生しました。エラーメッセージ: {e}\nコードを修正して再度実行してください。"})
            retry_count += 1

    print("コードの実行が完了しました。" if retry_count < 3 else "コードの修正に失敗しました。再実行回数が上限に達しました。")

Initialize prompt

In [ ]:
prompt_w_history = []

Execute

In [ ]:
%%_
kaggleのtitanicを分析してください

In [ ]:
%%_ train.csv
kaggleのtitanicを分析してください。対象のデータはtrain.csvです。

In [ ]:
%%_
可視化してください。